In [1]:
import pandas as pd
import numpy as np
import os
import sys
import pandas as pd
sys.path.append('..')
from tools import save_pickle

### Search in real disk

In [2]:
def case2path(x_p):
    cur_dirs = os.listdir(x_p)
    result = {}
    for dir_n in cur_dirs:
        case = dir_n[:12]
        if case in result:
            continue
        result[case] = os.path.join(x_p, dir_n)
    return pd.DataFrame.from_dict(result, orient='index', columns=['path'])

def merge_table_creator(selected_p, target_p='../data/Target.xlsx'):
    target = pd.read_excel(target_p)
    case_path_df = case2path(selected_p)
    merge_table = case_path_df.merge(target, left_index=True, right_on='sample')
    print(len(merge_table)/len(target))
    merge_table.reset_index(drop=True, inplace=True)
    return merge_table

In [3]:
merge_table = merge_table_creator('../../selected/')

0.9857142857142858


In [4]:
merge_table.head()

,path,sample,duration,observed
0,../../selected/TCGA-A1-A0SB-01Z-00-DX1,TCGA-A1-A0SB,259,False
1,../../selected/TCGA-A1-A0SD-01Z-00-DX1,TCGA-A1-A0SD,437,False
2,../../selected/TCGA-A1-A0SE-01Z-00-DX1,TCGA-A1-A0SE,1321,False
3,../../selected/TCGA-A1-A0SF-01Z-00-DX1,TCGA-A1-A0SF,1463,False
4,../../selected/TCGA-A1-A0SH-01Z-00-DX1,TCGA-A1-A0SH,1437,False


In [18]:
print(f'{len(merge_table)} cases are used')

759 cases are used


Use space to save time. I saved the image paths for each case in a dict.

In [13]:
def saving(merge_table):
    img_disk = {}
    img_count = {}
    for item in merge_table.iterrows():
        path = item[1][0]
        case = item[1][1]
        pool = os.listdir(path)
        img_disk[case] = tuple(os.path.abspath(os.path.join(path, f)) for f in pool)
        img_count[case] = len(pool)
    save_pickle(img_disk, '../../archive/', name='img_disk')
    return img_count

In [14]:
img_count = saving(merge_table)

Check the minimum of img number

In [16]:
from math import inf
def check_min(img_count):
    ceil = inf
    record = None
    for k, v in img_count.items():
        if v < ceil:
            ceil = v
            record = k
    print(f'{record} has the minimum number of imgs {ceil}')

In [17]:
check_min(img_count)

TCGA-A2-A0EQ has the minimum number of imgs 50


### Compare with raw slides

In [19]:
pixel_to_micron = 2.43 # 10X power

Next, combine slides metainfo and the outcome of Nesnet selection.

In [35]:
img_size = pd.read_csv('../data/power.txt', sep=r'\t', engine='python')
img_size['Name'] = img_size['Name'].str[:12]
img_size['Size'] = img_size['Size'].str.split(',')
img_sel = pd.DataFrame.from_dict(img_count, orient='index')
img_sel.columns = ['num']
img_comp = img_sel.merge(img_size, left_index=True, 
                         right_on='Name',how='inner').drop_duplicates('Name')

In [36]:
img_comp.head()

,num,Name,Power,Size
535,91,TCGA-A1-A0SB,40,"['82945', '112314']"
122,1046,TCGA-A1-A0SD,40,"['80287', '94075']"
414,5687,TCGA-A1-A0SE,40,"['73336', '122873']"
599,611,TCGA-A1-A0SF,40,"['84727', '65276']"
710,3441,TCGA-A1-A0SH,40,"['64427', '98874']"


Then, get useful information from raw data.

In [38]:
tile_area = 96 * 96
p_to_area = pixel_to_micron ** 2

img_comp['area_10X'] = img_comp['Size'].apply(
    lambda x:int(x[0].replace("'", ""))*int(x[1].replace("'", ""))
)/((img_comp['Power']/10)**2)

img_comp['ratio'] = tile_area * img_comp['num'] / img_comp['area_10X']
img_comp['ori_real_area'] = img_comp['area_10X'] * p_to_area
img_comp['sel_real_area'] = img_comp['ratio'] * img_comp['ori_real_area']

The case with the most selected image is TCGA-D8-A1XR. It's also the one with the largest seletion ratio.

In [39]:
img_comp.loc[img_comp['num'] == img_comp['num'].max()]

,num,Name,Power,Size,area_10X,ratio,ori_real_area,sel_real_area
74,27971,TCGA-D8-A1XR,40,"['90174', '81671']",4.602875e+08,0.560043,2.717952e+09,1.522169e+09


In [41]:
img_comp.loc[img_comp['ratio'] == img_comp['ratio'].max()]

,num,Name,Power,Size,area_10X,ratio,ori_real_area,sel_real_area
74,27971,TCGA-D8-A1XR,40,"['90174', '81671']",4.602875e+08,0.560043,2.717952e+09,1.522169e+09


In [42]:
img_comp.loc[img_comp['ratio'] == img_comp['ratio'].min()]

,num,Name,Power,Size,area_10X,ratio,ori_real_area,sel_real_area
511,60,TCGA-A8-A07E,20,"['110336', '77824']",2.146697e+09,0.000258,1.267603e+10,3265173.504


In [43]:
img_comp.loc[img_comp['num'] == img_comp['num'].min()]

,num,Name,Power,Size,area_10X,ratio,ori_real_area,sel_real_area
764,50,TCGA-A2-A0EQ,40,"['87178', '121613']",6.626236e+08,0.000695,3.912726e+09,2720977.92
695,50,TCGA-A8-A07O,20,"['58624', '42240']",6.190694e+08,0.000744,3.655543e+09,2720977.92
780,50,TCGA-A8-A08G,20,"['51456', '47872']",6.158254e+08,0.000748,3.636387e+09,2720977.92
651,50,TCGA-A8-A09R,20,"['65792', '32000']",5.263360e+08,0.000875,3.107961e+09,2720977.92
647,50,TCGA-C8-A26X,40,"['60051', '50456']",1.893708e+08,0.002433,1.118216e+09,2720977.92
290,50,TCGA-E9-A1NF,40,"['85372', '81394']",4.342980e+08,0.001061,2.564486e+09,2720977.92
188,50,TCGA-E9-A1NG,40,"['74666', '80437']",3.753693e+08,0.001228,2.216518e+09,2720977.92
174,50,TCGA-E9-A1NH,40,"['61330', '49794']",1.908666e+08,0.002414,1.127048e+09,2720977.92


I have clipped the minimal number to 50 to better train the model. So lots of cases are with 50 imgs selected.

### Summary

In [44]:
img_comp.describe()

,num,Power,area_10X,ratio,ori_real_area,sel_real_area
count,759.000000,759.000000,7.590000e+02,759.000000,7.590000e+02,7.590000e+02
mean,2977.586298,38.234519,5.382403e+08,0.057310,3.178255e+09,1.620389e+08
std,3381.354334,5.677602,2.857446e+08,0.061695,1.687293e+09,1.840118e+08
min,50.000000,20.000000,2.308479e+07,0.000258,1.363134e+08,2.720978e+06
25%,491.000000,40.000000,3.793671e+08,0.010588,2.240125e+09,2.672000e+07
50%,1874.000000,40.000000,5.176180e+08,0.039773,3.056482e+09,1.019823e+08
75%,4359.500000,40.000000,6.433840e+08,0.084139,3.799118e+09,2.372421e+08
max,27971.000000,40.000000,2.699035e+09,0.560043,1.593753e+10,1.522169e+09
